In [1]:
# Import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
# Create a variable with the URL to this tutorial
url = 'http://www.englishforum.ch/forum.php'
# Scrape the HTML at the url
r = requests.get(url)
# Turn the HTML into a Beautiful Soup object
soup = BeautifulSoup(r.text, 'lxml')
# soup

In [3]:
# Create four variables to score the scraped data in
dataframes={}
for link in soup.find_all("strong")[1:]:
    for linkhref in soup.find_all("a"):
        if(link in linkhref and not linkhref['href'].endswith('.html') and link.contents[0] not in dataframes):
            dataframes[link.contents[0]]=linkhref['href']
dataframes
#dataframes=sorted(dataframes)


{'Business & entrepreneur': 'http://www.englishforum.ch/business-entrepreneur/',
 'Commercial events': 'http://www.englishforum.ch/commercial-events/',
 'Complaints corner': 'http://www.englishforum.ch/complaints-corner/',
 'Concerts': 'http://www.englishforum.ch/concerts/',
 'Daily life': 'http://www.englishforum.ch/daily-life/',
 'Education': 'http://www.englishforum.ch/education/',
 'Employment': 'http://www.englishforum.ch/employment/',
 'Entertainment & dining': 'http://www.englishforum.ch/entertainment-dining/',
 'Family matters/health': 'http://www.englishforum.ch/family-matters-health/',
 'Finance/banking/taxation': 'http://www.englishforum.ch/finance-banking-taxation/',
 'Food and drink': 'http://www.englishforum.ch/food-drink/',
 'Housing in general': 'http://www.englishforum.ch/housing-general/',
 'Insurance': 'http://www.englishforum.ch/insurance/',
 'Introductions': 'http://www.englishforum.ch/introductions/',
 'Language corner': 'http://www.englishforum.ch/language-corner

In [4]:
from bs4.element import Tag

def page(url,index):
#     print(url,index)
    # Scrape the HTML at the url
    r = requests.get(url)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    subtopic={}
    for link in soup.find_all("table",id="threadslist"):
    #     print(link)
        index_aux=index
        dash=0
        for replies in link.find_all("td",class_="alt1"):
            if(replies.contents[0]!=' '):
                if(replies.contents[0]=='-'):
                    dash+=1
                subtopic[index_aux]=[]
                subtopic[index_aux].append(replies.contents[0])
# #                 print('rep',index_aux,replies.contents[0])

                index_aux+=1
        index_aux=index
        insertdash=0
        for views in link.find_all("td",class_="alt2"):
            if(views.contents[0]!=' '):
                if(views.contents[0]=='-'):
                    if(insertdash==0):
                        insertdash=1
                    else:
                        insertdash=0
#                         print('views',index_aux,views.contents[0])
                        subtopic[index_aux].append(views.contents[0])
                        index_aux+=1
                else:
#                     print('views',index_aux,views.contents[0])
#                 if(index_aux not in subtopic):
#                     print('errorrrrrrr',index_aux,views.contents[0])
#                     next
                    subtopic[index_aux].append(views.contents[0])
                    index_aux+=1        
#         index_aux=index
        for thread in link.find_all("a"):
            if('id' in thread.attrs):
#                 print('hrefs',index,link1.contents[0])
                subtopic[index].append(thread['href'])
                subtopic[index].append(thread.contents[0])
                index+=1
    return [subtopic,index]
    
    
def findAllPages(rootHTML):
    # Scrape the HTML at the url
    r = requests.get(rootHTML)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    lastUrl=''
    pages=[]
    pages.append(rootHTML)
    for link in soup.find_all("body"):
        for link1 in link.find_all("a"):
            if('title' in link1.attrs and 'Last' in link1['title']):
                lastUrl=link1['href']
                break
    if(lastUrl!=''):
        limit=re.findall(r'\d+',lastUrl)[0]
        for i in range(2,int(limit)):
            pages.append(rootHTML+"index"+str(i)+".html")
        pages.append(lastUrl)
        return pages
    return pages
    
def createDF(url):
#     print(url)
    index=0
    allsubtopics={}
    nextPages=findAllPages(url)
    for url in nextPages:
#         print('urlllll',url)
        result=page(url,index)
        allsubtopics.update(result[0])
        index=result[1]
    #         print('fin',subtopic)
    replies=[]
    views=[]
    hrefs=[]
    threads=[]
    for value in allsubtopics.values():
        replies.append(value[0])
        views.append(value[1])
        hrefs.append(value[2])
        threads.append(value[3])
    df = pd.DataFrame({'replies': replies, 'views': views, 'hrefs': hrefs, 'threads' : threads})
    return df

In [5]:
# for i in dataframes:
for index,i in enumerate(dataframes):
    print(index,i)

0 Education
1 Support
2 Leaving Switzerland
3 Introductions
4 Insurance
5 TV/internet/telephone
6 Transportation/driving
7 Permits/visas/government
8 Family matters/health
9 Travel/day trips/free time
10 Language corner
11 Swiss politics/news
12 Market Place
13 Entertainment & dining
14 Daily life
15 Finance/banking/taxation
16 Food and drink
17 Business & entrepreneur
18 Pet corner
19 Sports / Fitness / Beauty / Wellness
20 Off-Topic
21 Employment
22 Housing in general
23 Commercial events
24 Complaints corner
25 Other/general
26 Concerts
27 Social events
28 Swiss news via The Local


In [9]:
introductions=None
commercial_events=None
pet_corner=None
permits_visas_gov=None
housing=None
employment=None
sports_wellness=None
leaving_switzerland=None
food_drink=None
concerts=None
off_topic=None
business_enterpreteur=None
tv_internet_telephone=None
entertainment_dining=None
other_general=None
transportation_driving=None
complaints_corner=None
support=None
market_place=None
swiss_news=None
family_matters_health=None
social_events=None
insurance=None
language_corner=None
daily_life=None
finance_banking_taxation=None
swiss_politics_news=None
travel_free_time=None
education=None
df = pd.DataFrame()
for index,i in enumerate(sorted(dataframes)):
    print(index, dataframes[i])
    if index==0:  m=1
#         business_enterpreteur=createDF(dataframes[i])
#         business_enterpreteur.to_csv(i, sep='\t', encoding='utf-8',)
#         business_enterpreteur=[]
    elif index==1: m=0#commercial_events=createDF(dataframes[i])
        
        
    elif index==2:  m=1
#         complaints_corner=createDF(dataframes[i])   
#         complaints_corner.to_csv(i, sep='\t', encoding='utf-8')
#         complaints_corner=[]
    elif index==3: m=1 
#         concerts=createDF(dataframes[i])
#         concerts.to_csv(i, sep='\t', encoding='utf-8')
#         concerts=[]
    elif index==4:  m=1#daily_life=createDF(dataframes[i])
        
    elif index==5: m=0#education=createDF(dataframes[i])
        
    elif index==6: m=1#employment=createDF(dataframes[i])
        
    elif index==7: m=1#entertainment_dining=createDF(dataframes[i])
        
    elif index==8: m=1#family_matters_health=createDF(dataframes[i])
        
    elif index==9: m=1#finance_banking_taxation=createDF(dataframes[i])
        
    elif index==10: m=1#food_drink=createDF(dataframes[i])
        
    elif index==11: m=1#housing=createDF(dataframes[i])
        
    elif index==12: m=1#insurance=createDF(dataframes[i])
    
    elif index==13: m=1#introductions=createDF(dataframes[i])
        
    elif index==14: m=1#language_corner=createDF(dataframes[i])
        
    elif index==15: m=1#leaving_switzerland=createDF(dataframes[i])
        
    elif index==16: m=1#market_place=createDF(dataframes[i])
    
    elif index==17: m=1#off_topic=createDF(dataframes[i])
    
    elif index==18: m=1#other_general=createDF(dataframes[i])
    
    elif index==19: m=1#permits_visas_gov=createDF(dataframes[i])
    
    elif index==20: m=1#pet_corner=createDF(dataframes[i])
    
    elif index==21: m=0#social_events=createDF(dataframes[i])
    
    elif index==22: m=1#sports_wellness=createDF(dataframes[i])
    
    elif index==23: m=1#support=createDF(dataframes[i])

    elif index==24: m=0#swiss_news=createDF(dataframes[i])
    
#     elif index==25: swiss_politics_news=createDF(dataframes[i])
    
#     elif index==26: tv_internet_telephone=createDF(dataframes[i])
    
#     elif index==27: transportation_driving=createDF(dataframes[i])
    
#     else: travel_free_time=createDF(dataframes[i])
    else:
        df = pd.DataFrame()
        df=createDF(dataframes[i])
        df.to_csv(i.replace("/",""), sep='\t', encoding='utf-8',)
        del df # save memory ram
    
    
# other_general=createDF(list(dataframes.values())[0])
other_general

0 http://www.englishforum.ch/business-entrepreneur/
1 http://www.englishforum.ch/commercial-events/
2 http://www.englishforum.ch/complaints-corner/
3 http://www.englishforum.ch/concerts/
4 http://www.englishforum.ch/daily-life/
5 http://www.englishforum.ch/education/
6 http://www.englishforum.ch/employment/
7 http://www.englishforum.ch/entertainment-dining/
8 http://www.englishforum.ch/family-matters-health/
9 http://www.englishforum.ch/finance-banking-taxation/
10 http://www.englishforum.ch/food-drink/
11 http://www.englishforum.ch/housing-general/
12 http://www.englishforum.ch/insurance/
13 http://www.englishforum.ch/introductions/
14 http://www.englishforum.ch/language-corner/
15 http://www.englishforum.ch/leaving-switzerland/
16 http://www.englishforum.ch/market-place/
17 http://www.englishforum.ch/off-topic/
18 http://www.englishforum.ch/other-general/
19 http://www.englishforum.ch/permits-visas-government/
20 http://www.englishforum.ch/pet-corner/
21 http://www.englishforum.ch/so

In [10]:
df=pd.read_csv('Concerts',sep='\t',index_col=0)
df

,hrefs,replies,threads,views
0,http://www.englishforum.ch/concerts/50501-swit...,"1,100","Switzerland Gigs ""Heads Up""","146,057"
1,http://www.englishforum.ch/concerts/263903-any...,0,Any date-friendly little concerts in Zurich to...,241
2,http://www.englishforum.ch/concerts/258523-alt...,24,Alternative music anywhere?,"2,145"
3,http://www.englishforum.ch/concerts/258617-sym...,3,Symphony concerts for children?,769
4,http://www.englishforum.ch/concerts/262772-sma...,3,Small bars/places with acoustic livemusic?,587
5,http://www.englishforum.ch/concerts/199974-mam...,11,"Mamma Mia in Basel, English or German?","3,821"
6,http://www.englishforum.ch/concerts/262645-hul...,1,Hulk City -,392
7,http://www.englishforum.ch/concerts/245358-chr...,3,Christmas Carol concert (Basel) | 01.01.1970,"2,211"
8,http://www.englishforum.ch/concerts/261897-vol...,1,Volbeat,388
9,http://www.englishforum.ch/concerts/162168-bol...,7,Bollywood Movie (Jab Tak Hai Jaan) @ RiffRaff ...,"4,428"


In [525]:
def post(thread):
    # Scrape the HTML at the url
    r = requests.get(url)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    print(r.text)
    comments={}
#     print(soup.text)
#     for link in soup.get_text().find_all("td",{"class":"alt1"}):
#         for link2 in link.find_all("table"):
#         print(link.parent)
#         for link2 in link.find_all("div",{"id":"posts"})
#             print(link2)
#     for link in soup.find_all("div",{"align":"center"}):
#         for link2 in link.find_all("div",{"align":"left"}):
#             for link3 in link2.find_all("tr"):
#                 for link4 in link3.find_all("td",{"width":"99%"}):
#                     for link5 in link4.find_all("div"):
#                         print(link5)
#                     for link4 in link3.find_all("tbody"):
#                         for link5 in link4.find_all("tr"):
#                             print(link5.contents)
#         print('tableeee',link.attrs,'posts' in link.contents)
#         for link1 in link.find_all("div"):
#             print('diveee','posts' in link1.contents)
#             if('id' in link1.attrs and link1['id']=='posts'):
#         ,id="posts"):
#                 print(link)
        
post('http://www.englishforum.ch/other-general/168845-gun-community.html')

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"> <html xmlns="http://www.w3.org/1999/xhtml" dir="ltr" lang="en"> <head>  <meta http-equiv="Pragma" content="no-cache" /> <meta http-equiv="Expires" content="-1" /> <meta http-equiv="Cache-Control" content="no-cache" />  <meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" /> <meta name="generator" content="vBulletin 3.8.4" /> <meta name="keywords" content="English information Switzerland Zurich expat activities community forum chat meet foreigner" /> <meta name="description" content="Community portal for English-speaking people in Switzerland" />  <style type="text/css" id="vbulletin_css">
/**
* vBulletin 3.8.4 CSS
* Style: 'Default Style'; Style ID: 1
*/
body
{
	background: #ffffff;
	color: #000000;
	font: 10pt verdana, geneva, lucida, 'lucida grande', arial, helvetica, sans-serif;
	margin: 5px 10px 10px 10px;
	padding: 0px 25px;
}
a:link, body_al